In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import shark_clean as sh

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_df = pd.read_excel(url)

In [4]:
shark_df = shark_df.drop(['pdf', 'href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22','Unnamed: 11','Source'], axis=1)

In [5]:
shark_df = shark_df.dropna(how='all')

In [6]:
shark_df = shark_df.dropna(subset=['Country', 'State', 'Location'])

In [7]:
shark_df.columns = shark_df.columns.str.lower()

In [8]:
#standarizing the states in USA

usa_states = shark_df.loc[shark_df['country'] == 'USA', 'state']
usa_states_unique = usa_states.unique()
usa_states_unique

array(['Hawaii', 'Florida', 'California', 'South Carolina',
       'North Carolina', 'New York', 'New Jersey', 'Samoa', 'Louisiana',
       'Mississippi', 'Noirth Carolina', 'Georgia', 'Alabama', 'Texas',
       'Maryland', 'Maui', 'Oregon', 'Franklin County, Florida',
       'Virgin Islands', 'Maine', 'Bahamas', 'Delaware', 'Guam',
       'Cayman Islands', 'Rhode Island', 'Massachusetts', 'Washington',
       'Puerto Rico', 'Virginia', 'US Virgin Islands', 'Kentucky',
       'New Mexico', 'South Carolina ', 'Alaska', 'Missouri',
       'North Carolina ', 'Florida ', 'Connecticut', 'Pennsylvania',
       'Illinois', 'Wake Island', ' North Carolina', ' New Jersey',
       'New York ', 'CUBA'], dtype=object)

In [9]:
#Remove cuba

row_index_cuba = shark_df.index[shark_df['state'].str.lower() == 'cuba'].tolist()

# Update 'country' and 'state' in the identified row
shark_df.loc[row_index_cuba, 'country'] = 'CUBA'
shark_df.loc[row_index_cuba, 'state'] = ''

# Standardize the state names in USA
shark_df['state'] = shark_df['state'].str.strip().str.lower().replace({
    'noirth carolina': 'north carolina',
    'maui': 'hawaii',
    'franklin county, florida': 'florida',
    'cuba': ''
})

# Check the unique values in 'state' for USA after replacement
usa_states = shark_df.loc[shark_df['country'] == 'USA', 'state']
usa_states_unique_after = usa_states.unique()
usa_states_unique_after

array(['hawaii', 'florida', 'california', 'south carolina',
       'north carolina', 'new york', 'new jersey', 'samoa', 'louisiana',
       'mississippi', 'georgia', 'alabama', 'texas', 'maryland', 'oregon',
       'virgin islands', 'maine', 'bahamas', 'delaware', 'guam',
       'cayman islands', 'rhode island', 'massachusetts', 'washington',
       'puerto rico', 'virginia', 'us virgin islands', 'kentucky',
       'new mexico', 'alaska', 'missouri', 'connecticut', 'pennsylvania',
       'illinois', 'wake island'], dtype=object)

In [10]:
## performing cleaning using functions from above cell to create activity_grouped and updating type column


shark_df['activity_grouped'] = shark_df['activity'].apply(sh.clean_activity)
shark_df['type'] = shark_df['type'].apply(sh.clean_type)
shark_df['species '] = shark_df['species '].apply(sh.clean_species)
shark_df['fatal_bool'] = shark_df['injury'].apply(sh.clean_fatal)

In [11]:
# clean 'sex' column to M, F and UNDEFINED
shark_df.sex = shark_df.sex.fillna('Undefined')
shark_df.sex = shark_df.sex.apply(lambda sex:sex.strip().upper() if isinstance(sex,str) else sex)
shark_df.sex = shark_df.sex.replace(['LLI'],['UNDEFINED'])

# clean 'name' column to fill nan as 'Unknown'
shark_df.name = shark_df.name.fillna('Unknown')

In [12]:
shark_df_us = shark_df[shark_df['country']=='USA']
shark_df_us

,date,year,type,country,state,location,activity,name,sex,age,injury,time,species,activity_grouped,fatal_bool
2,30 Dec-2023,2023.0,Unprovoked,USA,hawaii,"Baby Beach, Maui",Surfing,Jason Carter,M,39,FATAL,11h12,Unknown,water_sport,1
16,05 Nov-2023,2023.0,Unprovoked,USA,florida,"Juno Beach, Palm Beach County",Swimming,Steven Reinhardt,M,66,Lacerations to right forearm,10h30,Unknown,swimming,0
21,25 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Pua’ena Point, Haleiwa, Oahu",Surfing,male,M,30,Bite to right thigh,15h05,Tiger Shark,water_sport,0
25,15 Oct 2023,2023.0,Unprovoked,USA,hawaii,"Hanalei Bay, Kauai",Surfing,Kevin Kanehe,M,50,Left leg and hand injured,16h00,Tiger Shark,water_sport,0
26,13 Oct-2023,2023.0,Unprovoked,USA,california,"Linda Mar, Pacific State Beach, San Mateo County",Surfing,male,M,52,Minor linjury to left lower leg,15h45,Unknown,water_sport,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6879,Before 1958,0.0,Unprovoked,USA,florida,"Palm Beach, Palm Beach County",Standing,Horton Chase,M,NaN,Abrasions & bruises hip to ankle,NaN,Unknown,others,0
6913,Before 1921,0.0,Unprovoked,USA,florida,"Gadsden Point, Tampa Bay",Fishing,James Kelley,M,NaN,2-inch lacerations,NaN,Unknown,fishing,0
6918,Before 17-Jul-1916,0.0,Unprovoked,USA,north carolina,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",M,NaN,"""Lost leg""",NaN,Unknown,swimming,0
6922,Circa 1862,0.0,Unprovoked,USA,hawaii,Puna,NaN,"A ""chiefess""",F,NaN,Ankle bitten,NaN,Unknown,others,0


In [13]:
# Change all strings to integers
shark_df_us['year'] = shark_df_us['year'].astype(int)

C:\Users\Nimrod\AppData\Local\Temp\ipykernel_36428\427528797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us['year'] = shark_df_us['year'].astype(int)


In [14]:
# Get the unique value of these cells
result = shark_df_us['year'].unique()
smaller_than_4 = [value for value in result if len(str(value)) < 4]
print(smaller_than_4)

[0]


In [15]:
# Replace these values with the Mode of the column year
for item in smaller_than_4:
    shark_df_us['year'] = shark_df_us['year'].replace(item, shark_df_us.year.mode()[0])

C:\Users\Nimrod\AppData\Local\Temp\ipykernel_36428\3655955125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us['year'] = shark_df_us['year'].replace(item, shark_df_us.year.mode()[0])


In [16]:
# Check again for irregularities
shark_df_us['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002,
       2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991,
       1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980,
       1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969,
       1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958,
       1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947,
       1945, 1944, 1943, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934,
       1933, 1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923,
       1922, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911,
       1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900,
       1899, 1898, 1897, 1896, 1895, 1894, 1892, 1891, 1890, 1889, 1887,
       1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878, 1876, 1874,
       1873, 1872, 1871, 1870, 1866, 1865, 1864, 18

In [17]:
# format the Dates in to the same format
shark_df_us["date"] = shark_df_us["date"].str.replace('  ', '-').str.replace(' ', '-').str.replace('--', '-').str.replace('Reported', '').str.replace('`', '').str.replace('.', '').str.strip()
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: '-'.join(filter(lambda x: x and len(x) <= 10, date_list)))
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: '-'.join(x.split('-')) if pd.notna(x) and len(x.split('-')) > 1 else x)
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: date_list[:1] + [date_list[1][:3]] + date_list[2:] if len(date_list) == 3 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01', 'Jan'] + date_list if len(date_list) == 1 and len(date_list[0]) == 4 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')
shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01'] + date_list if len(date_list) == 2 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')

C:\Users\Nimrod\AppData\Local\Temp\ipykernel_36428\497128711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us["date"] = shark_df_us["date"].str.replace('  ', '-').str.replace(' ', '-').str.replace('--', '-').str.replace('Reported', '').str.replace('`', '').str.replace('.', '').str.strip()
C:\Users\Nimrod\AppData\Local\Temp\ipykernel_36428\497128711.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_df_us["date"] = shark_df_us["date"].apply(lambda x: x.split('-') if pd.notna(x) else [])

In [18]:
shark_df_us.to_csv('C:\\Users\\Nimrod\\Documents\\GitHub\\2nd_try_iron_repo\\Quests\\shark_df_test.csv', index=False)
